In [2]:
import numpy as np
import pandas as pd
import pickle


from tensorflow.keras.models import load_model 


2024-10-14 14:18:35.825775: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-14 14:18:35.830140: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-14 14:18:35.851837: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-14 14:18:35.905395: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-14 14:18:35.918786: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-14 14:18:35.948673: I tensorflow/core/platform/cpu_feature_gu

In [25]:
# Load  the Model
model = load_model('model.h5')

# Load the LabelEncoder
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

# Load the OneHotEncoder
with open('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo = pickle.load(file)

# Load the Scalar
with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [66]:
# Input data

input_data = {
    'CreditScore' : 100,
    'Geography' : 'Spain',
    'Gender' : 'Male',
    'Age' : 25,
    'Tenure' : 4,
    'Balance' : 80000,
    'NumOfProducts' : 1,
    'HasCrCard' : 0,
    'IsActiveMember' : 0,
    'EstimatedSalary' : 80000
}

In [67]:
input_data = pd.DataFrame([input_data])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,100,Spain,Male,25,4,80000,1,0,0,80000


In [68]:
# Label_encoder 

input_data['Gender'] = label_encoder_gender.transform(input_data['Gender'])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,100,Spain,1,25,4,80000,1,0,0,80000


In [69]:
# Onehot_encoder 

geo_encoded = onehot_encoder_geo.transform([input_data['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

/home/magesh/anaconda3/envs/dl_env/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,0.0,0.0,1.0


In [70]:
input_data= pd.concat([input_data.reset_index(drop=True),geo_encoded_df],axis=1)
input_df= input_data.drop('Geography',axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,100,1,25,4,80000,1,0,0,80000,0.0,0.0,1.0


In [71]:
# Scaling the Input data

scaled_input = scaler.transform(input_df)
scaled_input

array([[-5.6926475 ,  0.91509065, -1.32660798, -0.35520275,  0.05670408,
        -0.91150957, -1.55337352, -1.02327069, -0.34988674, -0.99850112,
        -0.58312392,  1.74601919]])

In [73]:
# Predict the churn

predictions = model.predict(scaled_input)
predictions

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


array([[0.06900273]], dtype=float32)